### PySpark Joins

Data analysis usually entails working with multiple datasets or tables. Joining data frames is a common operation when working with related tables. There are multiple ways to join Dataframes (Tables).
- inner Join
- Left Outer Join
- Right Outer Join
- Full Outer Join
<br> 
At the most basic level, a join operation is a way to take the data from one data frame and link it to another one according to a set of rules

In [1]:
from pyspark.sql import SparkSession
spark = ( SparkSession.builder
         .appName('Table Joins')
         .getOrCreate()
        )

In [2]:
emp = [(1,"Smith",-1,"2018","10","M",3000), \
    (2,"Rose",1,"2010","20","M",4000), \
    (3,"Williams",1,"2010","10","M",1000), \
    (4,"Jones",2,"2005","10","F",2000), \
    (5,"Brown",2,"2010","40","",-1), \
      (6,"Brown",2,"2010","50","",-1) \
  ]
empColumns = ["emp_id","name","superior_emp_id","year_joined", \
       "emp_dept_id","gender","salary"]
empDF = spark.createDataFrame(data=emp, schema = empColumns)
empDF.printSchema()
empDF.show(truncate=False)

root
 |-- emp_id: long (nullable = true)
 |-- name: string (nullable = true)
 |-- superior_emp_id: long (nullable = true)
 |-- year_joined: string (nullable = true)
 |-- emp_dept_id: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- salary: long (nullable = true)

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |
|2     |Rose    |1              |2010       |20         |M     |4000  |
|3     |Williams|1              |2010       |10         |M     |1000  |
|4     |Jones   |2              |2005       |10         |F     |2000  |
|5     |Brown   |2              |2010       |40         |      |-1    |
|6     |Brown   |2              |2010       |50         |      |-1    |
+------+--------+---------------+-----------+-----------+------+-----

In [3]:
import pyspark.sql.functions as F
dept = [("Finance",10), \
    ("Marketing",20), \
    ("Sales",30), \
    ("IT",40) \
  ]
deptColumns = ["dept_name","dept_id"]
deptDF = spark.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+



## Recipe of Joins

We Have now two tables.
- empDF
- deptDF    
we are now ready to start __Joins__. A join operation require 03 ingredients:
1. Two tables, A __Left__ Table (empDF) and a __right__ table (deptDF)
2. One or More predicates (conditions which determine how records between table are joined)
3. Method to indicate how to Join on success of condition and what to do on failure of condition  
Every join operation in PySpark will follow the abovementioned same blueprint  
__[left_DF].join ( [Right_DF], on= [predicates], how= [method] )__

### The rules to a successful Joins: The Predicates

 - The predicates of a PySpark join are rules between columns of the left and right data frames
 - A join is performed record-wise.
 - each record on the left data frame is compared (via the predicates) to each record on the right data frame
     - If the predicates return True, the join is a match  (what happens will be dictated by 'Method')
     - if False then no-match (What happen will be dictated by Method)

### Our Example 

For our two data frames, we will build the predicate
***
empDF["emp_dept_id"] == deptDF['dept_id']. 
***
In plain English, this translates to “match the records from the empDF data frame to the records from the deptDF data frame
when the value of their **dept_id** column is equal.

There are two important points to highlight.
- If one record in Left *resolves predicate* with more than one record in right (vice versa),  this record will be duplicated in joined Table
- if one record in left or right tables *does not resolve* predicate with any record in other table, it will not be present in resulting table __UNLESS__ the join method specifies a method for failed predicates

In [5]:
print(f"Emp Count:{empDF.count()} -- Dept count : {deptDF.count()} ")

Emp Count:6 -- Dept count : 4 


In [6]:
empDF.join(deptDF, on = empDF["emp_dept_id"] == deptDF["dept_id"]).count()

5

In [4]:
empDF.join(deptDF, on = empDF["emp_dept_id"] == deptDF["dept_id"]).show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



we can use multiple conditions by separating them with Boolean operators such as | (or) or & (and)

In [9]:
#logs.join(log_idfr, (logs["LogServiceID"] == log_idfr["LogServiceID"]) and (logs["left_col"] < log_idfr["right_col"])  )

In [10]:
empDF.show()

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     5|   Brown|              2|       2010|         40|      |    -1|
|     6|   Brown|              2|       2010|         50|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+



In [11]:
deptDF.show()

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



### INNER JOIN

- Most Common Join
- PySpark defaults to inner join if No eplicit method is passed.
- returns a record if the predicate is true and drops it if false
- duplicate recorcd in **Left_table** if it matches more than one records of **Right_table**

In [5]:
empDF.join(deptDF, on=empDF["emp_dept_id"] == deptDF["dept_id"], how='inner').show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



- As mentioned, INNER join dropped 'emp_dept_id'=50 from empDF and 'dept_id'=30 from deptDF

### Left Join (Left Outer Join)

- Just like an INNER Join with only difference being:
   - what happens when predicate (condition) is False   
<br>   
<br>
- A **left join** adds unmatched records of **Left_table** in the Joined Table, filling columns from **Right_Table** with NULL

In [6]:
#leftouter
empDF.join(deptDF, on = empDF['emp_dept_id']== deptDF['dept_id'], how='leftouter').show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|   Brown|              2|       2010|         50|      |    -1|     null|   null|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



### Right Join (Right Outer Join)

- Just like an INNER Join with only difference being:
   - what happens when predicate (condition) is False   
<br>   
<br>
- A right join adds unmatched records of **Right_Table** in the Joined Table, filling columns from **Left_Table** with NULL    

In [7]:
#rightouter
empDF.join(deptDF, on=empDF['emp_dept_id']== deptDF['dept_id'], how='rightouter').show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  null|    null|           null|       null|       null|  null|  null|    Sales|     30|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



### Full Outer Join

- Simply a fusion of both left and right Join
- add unmatched records from both **Left_table** and **Right_Table** padding with NULL

In [8]:
#outer, full, fullouter
empDF.join(deptDF, on=empDF['emp_dept_id']== deptDF['dept_id'], how='full_outer').show()

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|  null|    null|           null|       null|       null|  null|  null|    Sales|     30|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
|     6|   Brown|              2|       2010|         50|      |    -1|     null|   null|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



### Left Semi-Join 

- Same as INNER Join except:
    - Keeps the columns of the left table
    - won’t duplicate the records in the left table
<br>
<br>
- Its main purpose is to filter records from a table based on a predicate that is depending on another table

In [19]:
empDF.join(deptDF, on=empDF['emp_dept_id']== deptDF['dept_id'], how='left_semi').show()
# The column deptDF['dept_id'] is NOT part of the results

+------+--------+---------------+-----------+-----------+------+------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+--------+---------------+-----------+-----------+------+------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|
|     3|Williams|              1|       2010|         10|     M|  1000|
|     4|   Jones|              2|       2005|         10|     F|  2000|
|     2|    Rose|              1|       2010|         20|     M|  4000|
|     5|   Brown|              2|       2010|         40|      |    -1|
+------+--------+---------------+-----------+-----------+------+------+



### Left Anti Join

- Opposite of INNER Join
    - keep only the records from **left_table** that **dont match** the predicate with any record in **right table**

In [9]:
empDF.join(deptDF, on=empDF['emp_dept_id']== deptDF['dept_id'], how='left_anti').show()

+------+-----+---------------+-----------+-----------+------+------+
|emp_id| name|superior_emp_id|year_joined|emp_dept_id|gender|salary|
+------+-----+---------------+-----------+-----------+------+------+
|     6|Brown|              2|       2010|         50|      |    -1|
+------+-----+---------------+-----------+-----------+------+------+



 ### Cross Join

 - returns a record for every record pair, regardless of the value the predicates return

In [11]:
empDF.join(deptDF, on=empDF['emp_dept_id']== deptDF['dept_id'], how='cross').show(20)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|    name|superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|     1|   Smith|             -1|       2018|         10|     M|  3000|  Finance|     10|
|     3|Williams|              1|       2010|         10|     M|  1000|  Finance|     10|
|     4|   Jones|              2|       2005|         10|     F|  2000|  Finance|     10|
|     2|    Rose|              1|       2010|         20|     M|  4000|Marketing|     20|
|     5|   Brown|              2|       2010|         40|      |    -1|       IT|     40|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+



## Spark SQL

In [11]:
empDF.createOrReplaceTempView("EMP")
deptDF.createOrReplaceTempView("DEPT")

joinDF = spark.sql("select * from EMP e, DEPT d where e.emp_dept_id == d.dept_id") \
  .show(truncate=False)

joinDF2 = spark.sql("select * from EMP e INNER JOIN DEPT d ON e.emp_dept_id == d.dept_id") \
  .show(truncate=False)

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|1     |Smith   |-1             |2018       |10         |M     |3000  |Finance  |10     |
|3     |Williams|1              |2010       |10         |M     |1000  |Finance  |10     |
|4     |Jones   |2              |2005       |10         |F     |2000  |Finance  |10     |
|2     |Rose    |1              |2010       |20         |M     |4000  |Marketing|20     |
|5     |Brown   |2              |2010       |40         |      |-1    |IT       |40     |
+------+--------+---------------+-----------+-----------+------+------+---------+-------+

+------+--------+---------------+-----------+-----------+------+------+---------+-------+
|emp_id|name    |superior_emp_id|year_joined|emp_dept_id|gender|salary|dept_name|dept_id|
+------+-